In [4]:
setwd("~/Upd-Germline-Genomics")
source("_targets.R")

Loading required package: viridisLite


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## S5 L2FE_Effect_TSS

Histone mark profile plot - local maxima and their significance to the "off" genes t-test.

Testing the main figure ChIC-ChIP average profile plots: This involves finding a good local maximum of smooth profile L2FE to test for enrichment (or we also may see depletion compared to a more uniform, closer to zero, "Q1" off gene profile). Extrema are detected using the second order finite difference (diff(diff())), and then we take the maximum value of the track. It is tested against the "Q1" off gene quantification average profile at this x point (in bp) of the TSS heatmap.

In [2]:
tar_load(matches("^chic.heatmap.tss_H3K(4|27|9)_(Germline|Somatic)_CN_chr"))
tar_load(c(quartile.factor_Germline, quartile.factor_Somatic))

In [3]:
all.equal(
  sort(match(rownames(chic.heatmap.tss_H3K27_Somatic_CN_chr), names(quartile.factor_Somatic))),
  seq_along(quartile.factor_Somatic)
)

[1] TRUE

In [4]:
data <- tibble(
  cross_join(
    tibble(
      celltype = c("Germline", "Germline", "Germline", "Somatic", "Somatic", "Somatic"),
      panel = c("H3K4me3", "H3K27me3", "H3K9me3", "H3K4me3", "H3K27me3", "H3K9me3"),
      heatmap = list(
        chic.heatmap.tss_H3K4_Germline_CN_chr,
        chic.heatmap.tss_H3K27_Germline_CN_chr,
        chic.heatmap.tss_H3K9_Germline_CN_chr,
        chic.heatmap.tss_H3K4_Somatic_CN_chr,
        chic.heatmap.tss_H3K27_Somatic_CN_chr,
        chic.heatmap.tss_H3K9_Somatic_CN_chr
      )
    ),
    tibble(
      rowname = c("low", "medium", "high"),
      level = c("Q2", "Q3", "Q4"),
    ),
  ),
  quant = mapply(
    \(celltype, heatmap) get(str_glue("quartile.factor_", celltype))[rownames(heatmap)],
    celltype,
    heatmap,
    SIMPLIFY=FALSE
  ),
  x = mapply(
    \(level, heatmap, quant) colnames(heatmap)[
      which.max(
        colMeans(heatmap[quant == level, 1:1000], na.rm=T) %>%
          replace(
            c(
              1,
              1 + which(
                diff(diff(colMeans(heatmap[quant == level, 1:1000], na.rm=T))) >= -1e-4
              ),
              1000
            ),
            0
          )
      )
    ],
    level,
    heatmap,
    quant
  ),
  ttest = mapply(
    \(heatmap, quant, level, x) t.test(
      heatmap[quant == level, x],
      heatmap[quant == "Q1", x]
    ),
    heatmap,
    quant,
    level,
    x,
    SIMPLIFY=FALSE
  ),
  t = sapply(ttest, \(obj) obj$statistic["t"]),
  p.adjust = p.adjust(
    sapply(
      ttest,
      \(obj) obj$p.value
    )
  ),
  signif = structure(
    cut(p.adjust, c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
    levels = c("****", "***", "**", "*", ""),
    class = "factor"
  )
) %>%
  subset(select=-c(heatmap,quant,ttest)) %>%
  print()

# A tibble: 18 × 8
   celltype panel    rowname level x           t  p.adjust signif
   <chr>    <chr>    <chr>   <chr> <chr>   <dbl>     <dbl> <fct> 
 1 Germline H3K4me3  low     Q2    131    24.1   1.51e-114 "****"
 2 Germline H3K4me3  medium  Q3    132    37.4   9.17e-242 "****"
 3 Germline H3K4me3  high    Q4    132    40.1   4.40e-271 "****"
 4 Germline H3K27me3 low     Q2    133    -2.82  1.92e-  2 "*"   
 5 Germline H3K27me3 medium  Q3    133    -1.93  1.59e-  1 ""    
 6 Germline H3K27me3 high    Q4    -30    -1.49  2.71e-  1 ""    
 7 Germline H3K9me3  low     Q2    -14     4.37  6.35e-  5 "****"
 8 Germline H3K9me3  medium  Q3    -10     6.52  6.64e- 10 "****"
 9 Germline H3K9me3  high    Q4    -10     5.29  9.92e-  7 "****"
10 Somatic  H3K4me3  low     Q2    308     9.31  1.70e- 19 "****"
11 Somatic  H3K4me3  medium  Q3    134    10.7   2.91e- 25 "****"
12 Somatic  H3K4me3  high    Q4    135     8.96  3.91e- 18 "****"
13 Somatic  H3K27me3 low     Q2    483     5.24  1.16e-  

## S6 Occupancy_Effect_TSS

Recapitulate the above Welch's T-Test but for the H3 ChIC-seq pileup (nucleosome occupancy).

In [6]:
tar_load(
  c(
    chic.heatmap.tss.nucleosome_H3K27_Germline_CN_chr,
    chic.heatmap.tss.nucleosome_H3K27_Somatic_CN_chr
  )
)

In [7]:
data <- tibble(
  cross_join(
    tibble(
      celltype = c("Germline", "Somatic"),
      heatmap = list(
        chic.heatmap.tss.nucleosome_H3K27_Germline_CN_chr,
        chic.heatmap.tss.nucleosome_H3K27_Somatic_CN_chr
      )
    ),
    tibble(
      rowname = c("low", "medium", "high"),
      level = c("Q2", "Q3", "Q4"),
    ),
  ),
  quant = mapply(
    \(celltype, heatmap) get(str_glue("quartile.factor_", celltype))[rownames(heatmap)],
    celltype,
    heatmap,
    SIMPLIFY=FALSE
  ),
  x = mapply(
    \(level, heatmap, quant) colnames(heatmap)[
      which.max(
        colMeans(heatmap[quant == level, 1:1000], na.rm=T) %>%
          replace(
            c(
              1,
              1 + which(
                diff(diff(colMeans(heatmap[quant == level, 1:1000], na.rm=T))) >= -1e-4
              ),
              1000
            ),
            0
          )
      )
    ],
    level,
    heatmap,
    quant
  ),
  ttest = mapply(
    \(heatmap, quant, level, x) t.test(
      heatmap[quant == level, x],
      heatmap[quant == "Q1", x]
    ),
    heatmap,
    quant,
    level,
    x,
    SIMPLIFY=FALSE
  ),
  t = sapply(ttest, \(obj) obj$statistic["t"]),
  p.adjust = p.adjust(
    sapply(
      ttest,
      \(obj) obj$p.value
    )
  ),
  signif = structure(
    cut(p.adjust, c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
    levels = c("****", "***", "**", "*", ""),
    class = "factor"
  )
) %>%
  subset(select=-c(heatmap,quant,ttest)) %>%
  print()

# A tibble: 6 × 7
  celltype rowname level x         t  p.adjust signif
  <chr>    <chr>   <chr> <chr> <dbl>     <dbl> <fct> 
1 Germline low     Q2    131    29.9 7.84e-168 ****  
2 Germline medium  Q3    133    47.1 0         ****  
3 Germline high    Q4    134    46.5 0         ****  
4 Somatic  low     Q2    129    34.1 4.06e-208 ****  
5 Somatic  medium  Q3    131    42.2 2.22e-290 ****  
6 Somatic  high    Q4    133    38.2 1.02e-248 ****  


## Repli Quartile - Average Gene Profile Histone Mark T-Test

Histone mark code & replication regime profile plot - local maxima and their significance to the "off" genes t-test

Although these versions of the Low / Medium / High (ML - EM - E timing) gene profiles and their analysis are not shown in a table, these analyses support Figure 6S - E-F.

In [8]:
tar_load(c(repli.gene_Germline, repli.gene_Somatic))
table(repli.gene_Germline)

repli.gene_Germline
   L   ML   EM    E 
 672 3378 5503 7999 

In [9]:
all.equal(
  sort(match(rownames(chic.heatmap.tss_H3K27_Somatic_CN_chr), names(repli.gene_Somatic))),
  seq_along(repli.gene_Somatic)
)

[1] TRUE

In [10]:
data <- tibble(
  cross_join(
    tibble(
      celltype = c("Germline", "Germline", "Germline", "Somatic", "Somatic", "Somatic"),
      panel = c("H3K4me3", "H3K27me3", "H3K9me3", "H3K4me3", "H3K27me3", "H3K9me3"),
      heatmap = list(
        chic.heatmap.tss_H3K4_Germline_CN_chr,
        chic.heatmap.tss_H3K27_Germline_CN_chr,
        chic.heatmap.tss_H3K9_Germline_CN_chr,
        chic.heatmap.tss_H3K4_Somatic_CN_chr,
        chic.heatmap.tss_H3K27_Somatic_CN_chr,
        chic.heatmap.tss_H3K9_Somatic_CN_chr
      )
    ),
    tibble(
      level = c("ML", "EM", "E")
    ),
  ),
  quant = mapply(
    \(celltype, heatmap) get(str_glue("repli.gene_", celltype))[rownames(heatmap)],
    celltype,
    heatmap,
    SIMPLIFY=FALSE
  ),
  x = mapply(
    \(level, heatmap, quant) colnames(heatmap)[
      which.max(
        colMeans(heatmap[quant == level, 1:1000], na.rm=T) %>%
          replace(
            c(
              1,
              1 + which(
                diff(diff(colMeans(heatmap[quant == level, 1:1000], na.rm=T))) >= -1e-4
              ),
              1000
            ),
            0
          )
      )
    ],
    level,
    heatmap,
    quant
  ) %>%
    replace(
      celltype == "Somatic" & panel == "H3K9me3" & level == "EM", -14
    ),
  ttest = mapply(
    \(heatmap, quant, level, x) t.test(
      heatmap[quant == level, x],
      heatmap[quant == "L", x]
    ),
    heatmap,
    quant,
    level,
    x,
    SIMPLIFY=FALSE
  ),
  t = sapply(ttest, \(obj) obj$statistic["t"]),
  p.adjust = p.adjust(
    sapply(
      ttest,
      \(obj) obj$p.value
    )
  ),
  signif = structure(
    cut(p.adjust, c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
    levels = c("****", "***", "**", "*", ""),
    class = "factor"
  )
) %>%
  mutate(panel = factor(panel, str_glue("H3K{c(4,27,9)}me3")), level = factor(level, c("ML", "EM", "E"))) %>%
  arrange(celltype, panel, level) %>%
  subset(select=-c(heatmap,quant,ttest)) %>%
  print()

# A tibble: 18 × 7
   celltype panel    level x          t p.adjust signif
   <chr>    <fct>    <fct> <chr>  <dbl>    <dbl> <fct> 
 1 Germline H3K4me3  ML    136    3.42  6.42e- 3 "**"  
 2 Germline H3K4me3  EM    131   14.9   2.30e-45 "****"
 3 Germline H3K4me3  E     134   19.8   1.24e-72 "****"
 4 Germline H3K27me3 ML    146    1.78  3.76e- 1 ""    
 5 Germline H3K27me3 EM    133    3.27  9.62e- 3 "**"  
 6 Germline H3K27me3 E     136    3.48  5.62e- 3 "**"  
 7 Germline H3K9me3  ML    -9     2.22  2.11e- 1 ""    
 8 Germline H3K9me3  EM    -8     1.91  3.35e- 1 ""    
 9 Germline H3K9me3  E     -10    4.15  4.40e- 4 "***" 
10 Somatic  H3K4me3  ML    133    2.05  2.86e- 1 ""    
11 Somatic  H3K4me3  EM    135    8.54  2.45e-16 "****"
12 Somatic  H3K4me3  E     130    6.38  2.69e- 9 "****"
13 Somatic  H3K27me3 ML    493    0.632 1   e+ 0 ""    
14 Somatic  H3K27me3 EM    491    0.776 1   e+ 0 ""    
15 Somatic  H3K27me3 E     480    0.894 1   e+ 0 ""    
16 Somatic  H3K9me3  ML    41

## S5 - Celltype_Effect at 50% of gene CDS

Histone mark at the gene midpoint - Welch's t-test

In [73]:
tar_load(matches("^chic.heatmap.paneled_H3K(4|27)_(Germline|Somatic)_CN_chr"))

In [79]:
data <- tribble(
  ~mark, ~germline, ~somatic,
  "H3K4me3", chic.heatmap.paneled_H3K4_Germline_CN_chr, chic.heatmap.paneled_H3K4_Somatic_CN_chr,
  "H3K27me3", chic.heatmap.paneled_H3K27_Germline_CN_chr, chic.heatmap.paneled_H3K27_Somatic_CN_chr,
) %>%
  rowwise() %>%
  reframe(
    mark,
    ttest = t.test(
      germline[
        names(quartile.factor_Germline)[quartile.factor_Germline != "Q1"],
        "50%"
      ],
      somatic[
        names(quartile.factor_Somatic)[quartile.factor_Somatic != "Q1"],
        "50%"
      ]
    ) %>%
      list(),
    x = "50%",
    t = ttest$statistic["t"],
    p.adjust = ttest$p.value,
  ) %>%
  mutate(
    p.adjust = p.adjust(p.adjust),
    signif = structure(
      cut(p.adjust, c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
      levels = c("****", "***", "**", "*", ""),
      class = "factor"
    )
  ) %>%
  subset(select = -ttest) %>%
  print()

# A tibble: 2 x 5
  mark     x           t p.adjust signif
  <chr>    <chr>   <dbl>    <dbl> <fct> 
1 H3K4me3  50%     0.105 9.17e- 1 ""    
2 H3K27me3 50%   -20.0   2.39e-87 "****"


## S5 L2FE_All_TE Summary of quantified transposable elements

Transposable elements selected for the violin plot and their L2FC paired t-test

In [82]:
tar_load(
  enriched.transposable.elements.peakcalling.broad.masked
)
H3K4_Germline <- tar_read(chic.experiment.quantify_H3K4_Germline_peakcalling.broad_masked)[enriched.transposable.elements.peakcalling.broad.masked]
H3K4_Somatic <- tar_read(chic.experiment.quantify_H3K4_Somatic_peakcalling.broad_masked)[enriched.transposable.elements.peakcalling.broad.masked]
H3K27_Germline <- tar_read(chic.experiment.quantify_H3K27_Germline_peakcalling.broad_masked)[enriched.transposable.elements.peakcalling.broad.masked]
H3K27_Somatic <- tar_read(chic.experiment.quantify_H3K27_Somatic_peakcalling.broad_masked)[enriched.transposable.elements.peakcalling.broad.masked]
H3K9_Germline <- tar_read(chic.experiment.quantify_H3K9_Germline_peakcalling.broad_masked)[enriched.transposable.elements.peakcalling.broad.masked]
H3K9_Somatic <- tar_read(chic.experiment.quantify_H3K9_Somatic_peakcalling.broad_masked)[enriched.transposable.elements.peakcalling.broad.masked]

The criteria are also found in the _targets build targets (H3 quantification >= 0.5 in all tracks so that the TE is present in all chromatin analyses)

In [ ]:
plot_features <- which(
    H3K4_Germline$score.molH3 >= 0.5 &
      H3K27_Germline$score.molH3 >= 0.5 &
      H3K9_Germline$score.molH3 >= 0.5 &
      H3K4_Somatic$score.molH3 >= 0.5 &
      H3K27_Somatic$score.molH3 >= 0.5 &
      H3K9_Somatic$score.molH3 >= 0.5
  )
data <- tribble(
  ~mark, ~germline, ~somatic,
  "H3K4me3", H3K4_Germline$L2FC[plot_features], H3K4_Somatic$L2FC[plot_features],
  "H3K27me3", H3K27_Germline$L2FC[plot_features], H3K27_Somatic$L2FC[plot_features],
  "H3K9me3", H3K9_Germline$L2FC[plot_features], H3K9_Somatic$L2FC[plot_features],
) %>%
  rowwise() %>%
  reframe(
    mark,
    ttest = t.test(germline, somatic, paired=T) %>% list(),
    t = ttest$statistic["t"],
    p.adjust = ttest$p.value,
  ) %>%
  mutate(
    p.adjust = p.adjust(p.adjust),
    signif = structure(
      cut(p.adjust, c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
      levels = c("****", "***", "**", "*", ""),
      class = "factor"
    )
  ) %>%
  print()

# A tibble: 3 x 5
  mark     ttest       t p.adjust signif
  <chr>    <list>  <dbl>    <dbl> <fct> 
1 H3K4me3  <htest>  3.50 1.30e- 3 **    
2 H3K27me3 <htest> -2.40 1.79e- 2 *     
3 H3K9me3  <htest>  8.55 1.52e-13 ****  


## Supplemental LOESS Model (Gene TSS)

These LOESS predictions support Figure 3S - C & E. We predict L2FE conditioned on the logCPM of the gene.

In [1]:
tar_load(
  c(
    chic.experiment.quantify.smooth_bw40_H3K4_Germline_CN_chr,
    chic.experiment.quantify.smooth_bw40_H3K27_Germline_CN_chr,
    chic.experiment.quantify.smooth_bw40_H3K9_Germline_CN_chr,
    chic.experiment.quantify.smooth_bw40_H3K4_Somatic_CN_chr,
    chic.experiment.quantify.smooth_bw40_H3K27_Somatic_CN_chr,
    chic.experiment.quantify.smooth_bw40_H3K9_Somatic_CN_chr,
    chic.gene.tss.diameter_40_chr
  )
)

ERROR: Error in tar_load(c(chic.experiment.quantify.smooth_bw40_H3K4_Germline_CN_chr, : could not find function "tar_load"


In [5]:
tar_load(
  c(
    chic.gene.enrichment.l2fc,
    Upd_cpm
  )
)
marks <- data.frame(
  celltype = c("Germline", "Somatic") %>% factor(., .) %>% rep(each = 3 * nrow(Upd_cpm)),
  mark = c("H3K4me3", "H3K27me3", "H3K9me3") %>% factor(., .) %>% rep(each = nrow(Upd_cpm)),
  melt(chic.gene.enrichment.l2fc[7:12], variable.name = "group", value.name = "L2FC"),
  logCPM = c(
    rep(log(Upd_cpm[, "germline"]) / log(10), 3) %>%
      replace(!is.finite(.), NA),
    rep(log(Upd_cpm[, "somatic"]) / log(10), 3) %>%
      replace(!is.finite(.), NA)
  ),
  color = c(
    rep(quartile.factor_Germline, 3),
    rep(quartile.factor_Somatic, 3)
  ) %>%
    structure(levels = c("off", "low", "medium", "high"), class = "factor")
) %>%
  tibble() %>%
  print()

No id variables; using all as measure variables



# A tibble: 105,348 x 6
   celltype mark    group            L2FC   logCPM color 
   <fct>    <fct>   <fct>           <dbl>    <dbl> <fct> 
 1 Germline H3K4me3 H3K4_Germline NA       2.36    high  
 2 Germline H3K4me3 H3K4_Germline -0.450   0.374   off   
 3 Germline H3K4me3 H3K4_Germline  0.298   0.00598 off   
 4 Germline H3K4me3 H3K4_Germline  0.768   1.40    medium
 5 Germline H3K4me3 H3K4_Germline  0.210  NA       off   
 6 Germline H3K4me3 H3K4_Germline -1.03    0.806   low   
 7 Germline H3K4me3 H3K4_Germline  0.0912  0.328   off   
 8 Germline H3K4me3 H3K4_Germline  0.443  -0.294   off   
 9 Germline H3K4me3 H3K4_Germline -0.0860  0.0128  off   
10 Germline H3K4me3 H3K4_Germline  0.0784  0.794   low   
# i 105,338 more rows


In [20]:
library(ggplot2)
xlim <- c(-3, 3)
ylim <- c(-3, 5)
markloess <- marks %>%
  group_by(group) %>%
  reframe(
    y = seq(min(ylim), max(ylim), by=0.01) %>%
      round(2) %>%
      subset(between(., logCPM %>% subset(. > -6) %>% quantile(0.025), logCPM %>% subset(. > -6) %>% quantile(0.975))),
    predict(
      loess(L2FC ~ logCPM),
      newdata=y,
      se = TRUE
    ) %>%
      as_tibble() %>%
      reframe(L2FC = fit, L2FC.se = se.fit),
    logCPM = y
  ) %>%
  print(n = 3)
ggplot(markloess, aes(y, L2FC, color=group)) + geom_line()

In [21]:
log10 <- \(v) log(v) / log(10)
x <- mapply(
  \(cpm, level) cpm %>%
    split(level) %>%
    sapply(\(v) v %>% subset(v != 0) %>% median %>% log10 %>% round(digits=2)),
  as.data.frame(Upd_cpm[, 1:2]),
  list(quartile.factor_Germline, quartile.factor_Somatic)
)
data <- markloess %>%
  group_by(group) %>%
  reframe(
    celltype = tolower(str_extract(group[1], "Germline|Somatic")),
    tibble(name = c("Q2", "Q3", "Q4")) %>%
      rowwise() %>%
      mutate(
        teststat = (
          L2FC[match(x[name, celltype], y)] -
            L2FC[match(x["Q1", celltype], y)]
        ) /
          sqrt(
            L2FC.se[match(x[name, celltype], y)]^2 +
              L2FC.se[match(x["Q1", celltype], y)]
          ),
        p.adjust = teststat %>%
          abs() %>%
          pnorm(lower.tail = F) %>%
          `*`(2) %>%
          pmin(1)
      )
  ) %>%
  subset(select = c(1, 3, 4, 5)) %>%
  mutate(
    p.adjust = p.adjust(p.adjust),
    signif = structure(
      cut(p.adjust, c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
      levels = c("****", "***", "**", "*", ""),
      class = "factor"
    )
  ) %>%
  print()

# A tibble: 18 x 5
   group          name  teststat p.adjust signif
   <fct>          <chr>    <dbl>    <dbl> <fct> 
 1 H3K4_Germline  Q2       4.26  2.07e- 4 "***" 
 2 H3K4_Germline  Q3       6.11  1.43e- 8 "****"
 3 H3K4_Germline  Q4       7.02  3.31e-11 "****"
 4 H3K27_Germline Q2       0.470 1   e+ 0 ""    
 5 H3K27_Germline Q3       0.599 1   e+ 0 ""    
 6 H3K27_Germline Q4       0.336 1   e+ 0 ""    
 7 H3K9_Germline  Q2      -1.94  2.59e- 1 ""    
 8 H3K9_Germline  Q3      -2.85  2.65e- 2 "*"   
 9 H3K9_Germline  Q4      -3.64  2.49e- 3 "**"  
10 H3K4_Somatic   Q2       5.82  6.97e- 8 "****"
11 H3K4_Somatic   Q3       7.46  1.52e-12 "****"
12 H3K4_Somatic   Q4       7.85  7.29e-14 "****"
13 H3K27_Somatic  Q2      -1.84  2.65e- 1 ""    
14 H3K27_Somatic  Q3      -3.63  2.49e- 3 "**"  
15 H3K27_Somatic  Q4      -5.94  3.62e- 8 "****"
16 H3K9_Somatic   Q2      -3.60  2.49e- 3 "**"  
17 H3K9_Somatic   Q3      -5.44  5.79e- 7 "****"
18 H3K9_Somatic   Q4      -7.08  2.30e-11 "****"


## S4 Cell_Type_Specific_Chromatin

Relative Enrichment (Quadrant II and IV) contingency table analysis.

The significance comes from a "simulation" of permuting Bernoulli (50/50) GSC-specific vs CySC-specific labels to the specific chromatin. It is actually an exact test where p-values come from the Binomial distribution which is the sum of these Bernoulli random variables for all of the sliding windows of chromatin.

In [6]:
library(dplyr)
library(GenomicRanges)

tar_load(chic.tile.diameter_500_chr)
tar_load(chromosome_pericetromere_label)

subset(chic.tile.diameter_500_chr, seqnames == "rDNA")[372:767] %>%
  reduce() %>%
  ranges()

analyze_loci <- c(
  seqnames(chic.tile.diameter_500_chr) %in% names(chr.lengths)
) %>%
  replace(
    which(seqnames(chic.tile.diameter_500_chr) == "rDNA")[372:767],
    TRUE
  )
seqnames <- seqnames(chic.tile.diameter_500_chr)[analyze_loci] %>%
  droplevels()
seqnames
gr <- chic.tile.diameter_500_chr[analyze_loci]
gr$region <- seqnames %>%
  as.character() %>%
  paste0(
    rep("", length(.)) %>%
      replace(
        to(findOverlaps(chromosome_pericetromere_label, gr)),
        "C"
      )
  )
table(gr$region)
gr$region <- gr$region %>%
  factor(c("2L", "2LC", "2RC", "2R", "3L", "3LC", "3RC", "3R", "4", "X", "Y", "rDNA"))

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:dplyr’:

    first, rename


The following object is masked from ‘package:utils’:

    findMatches


The following objects are mask

IRanges object with 1 range and 0 metadata columns:
          start       end     width
      <integer> <integer> <integer>
  [1]     36901     76900     40000

factor-Rle of length 1375880 with 8 runs
  Lengths: 235138 252870 281103 320794  13482 235423  36674    396
  Values :   2L     2R     3L     3R     4      X      Y      rDNA
Levels(8): 2L 2R 3L 3R 4 X Y rDNA


    2L    2LC     2R    2RC     3L    3LC     3R    3RC      4   rDNA      X 
221922  13216 196354  56516 231539  49564 278500  42294  13482    396 235423 
     Y 
 36674 

In [7]:
tar_load(chic.experiment.quantify_H3K4_Germline_peakcalling.broad_chr)
tar_load(chic.experiment.quantify_H3K4_Somatic_peakcalling.broad_chr)
chic.experiment.quantify_H3K4_Germline_peakcalling.broad_chr <-
  chic.experiment.quantify_H3K4_Germline_peakcalling.broad_chr[analyze_loci]
seqlevels(chic.experiment.quantify_H3K4_Germline_peakcalling.broad_chr) <-
  levels(seqnames)
seqnames(chic.experiment.quantify_H3K4_Germline_peakcalling.broad_chr) <-
  seqnames
chic.experiment.quantify_H3K4_Somatic_peakcalling.broad_chr <-
  chic.experiment.quantify_H3K4_Somatic_peakcalling.broad_chr[analyze_loci]
seqlevels(chic.experiment.quantify_H3K4_Somatic_peakcalling.broad_chr) <-
  levels(seqnames)
seqnames(chic.experiment.quantify_H3K4_Somatic_peakcalling.broad_chr) <-
  seqnames

tar_load(chic.experiment.quantify_H3K27_Germline_peakcalling.broad_chr)
tar_load(chic.experiment.quantify_H3K27_Somatic_peakcalling.broad_chr)
chic.experiment.quantify_H3K27_Germline_peakcalling.broad_chr <-
  chic.experiment.quantify_H3K27_Germline_peakcalling.broad_chr[analyze_loci]
seqlevels(chic.experiment.quantify_H3K27_Germline_peakcalling.broad_chr) <-
  levels(seqnames)
seqnames(chic.experiment.quantify_H3K27_Germline_peakcalling.broad_chr) <-
  seqnames
chic.experiment.quantify_H3K27_Somatic_peakcalling.broad_chr <-
  chic.experiment.quantify_H3K27_Somatic_peakcalling.broad_chr[analyze_loci]
seqlevels(chic.experiment.quantify_H3K27_Somatic_peakcalling.broad_chr) <-
  levels(seqnames)
seqnames(chic.experiment.quantify_H3K27_Somatic_peakcalling.broad_chr) <-
  seqnames

tar_load(chic.experiment.quantify_H3K9_Germline_peakcalling.broad_chr)
tar_load(chic.experiment.quantify_H3K9_Somatic_peakcalling.broad_chr)
chic.experiment.quantify_H3K9_Germline_peakcalling.broad_chr <-
  chic.experiment.quantify_H3K9_Germline_peakcalling.broad_chr[analyze_loci]
seqlevels(chic.experiment.quantify_H3K9_Germline_peakcalling.broad_chr) <-
  levels(seqnames)
seqnames(chic.experiment.quantify_H3K9_Germline_peakcalling.broad_chr) <-
  seqnames
chic.experiment.quantify_H3K9_Somatic_peakcalling.broad_chr <-
  chic.experiment.quantify_H3K9_Somatic_peakcalling.broad_chr[analyze_loci]
seqlevels(chic.experiment.quantify_H3K9_Somatic_peakcalling.broad_chr) <-
  levels(seqnames)
seqnames(chic.experiment.quantify_H3K9_Somatic_peakcalling.broad_chr) <-
  seqnames

In [8]:
chic.experiment.quantify_H3K4_Germline_peakcalling.broad_chr <-
  chic.experiment.quantify_H3K4_Germline_peakcalling.broad_chr %>%
    subset(seqnames(.) != "rDNA")
chic.experiment.quantify_H3K4_Somatic_peakcalling.broad_chr <-
  chic.experiment.quantify_H3K4_Somatic_peakcalling.broad_chr %>%
    subset(seqnames(.) != "rDNA")

chic.experiment.quantify_H3K27_Germline_peakcalling.broad_chr <-
  chic.experiment.quantify_H3K27_Germline_peakcalling.broad_chr %>%
    subset(seqnames(.) != "rDNA")
chic.experiment.quantify_H3K27_Somatic_peakcalling.broad_chr <-
  chic.experiment.quantify_H3K27_Somatic_peakcalling.broad_chr %>%
    subset(seqnames(.) != "rDNA")

chic.experiment.quantify_H3K9_Germline_peakcalling.broad_chr <-
  chic.experiment.quantify_H3K9_Germline_peakcalling.broad_chr %>%
    subset(seqnames(.) != "rDNA")
chic.experiment.quantify_H3K9_Somatic_peakcalling.broad_chr <-
  chic.experiment.quantify_H3K9_Somatic_peakcalling.broad_chr %>%
    subset(seqnames(.) != "rDNA")

gr <- gr %>% subset(seqnames(.) != "rDNA")

In [10]:
# Exact counts of Cell Type Specific Chromatin (in sliding windows of 100 bp)
contingency_data <- (
  tribble(
    ~mark, ~Germline, ~Somatic,
    "H3K4me3", chic.experiment.quantify_H3K4_Germline_peakcalling.broad_chr, chic.experiment.quantify_H3K4_Somatic_peakcalling.broad_chr,
    "H3K27me3", chic.experiment.quantify_H3K27_Germline_peakcalling.broad_chr, chic.experiment.quantify_H3K27_Somatic_peakcalling.broad_chr,
    "H3K9me3", chic.experiment.quantify_H3K9_Germline_peakcalling.broad_chr, chic.experiment.quantify_H3K9_Somatic_peakcalling.broad_chr,
  ) %>%
    group_by(mark) %>%
    reframe(
      region = gr$region,
      across(
        c(Germline, Somatic),
        \(gr) gr[[1]]$L2FC
      )
    ) %>%
    group_by(mark, region) %>%
    reframe(
      table = rbind(
        c(
          sum(between(Germline, -7.5, -0.2) & between(Somatic, 0.2, 7.5)),
          sum(between(Germline, 0.2, 7.5) & between(Somatic, 0.2, 7.5))
        ),
        c(
          sum(between(Germline, -7.5, -0.2) & between(Somatic, -7.5, -0.2)),
          sum(between(Germline, 0.2, 7.5) & between(Somatic, -7.5, -0.2))
        )
      )
    )
) %>%
  print()

# A tibble: 66 × 3
   mark     region table[,1]  [,2]
   <chr>    <fct>      <int> <int>
 1 H3K27me3 2L          5879 46836
 2 H3K27me3 2L         43264  8036
 3 H3K27me3 2LC          167   827
 4 H3K27me3 2LC         3838   543
 5 H3K27me3 2RC         1119  4090
 6 H3K27me3 2RC        16205  2946
 7 H3K27me3 2R          3932 55482
 8 H3K27me3 2R         29721  9261
 9 H3K27me3 3L          6498 51614
10 H3K27me3 3L         45468  8563
# ℹ 56 more rows


Exact Test

To find individuals of statistically independent quantification (no sliding overlap), we take the ratio of the window width (500 bp) to sliding window step (which we have set to 100 bp for this analysis). Thus, in the row entries of GSC-Specific chromatin window count and CySC-Specific chromatin window count, by dividing by this factor, we will be counting statistically independent individuals (non-overlapping chromatin, which also passes a MAPQ filter of 20 or more, further enhancing the accuracy of this test).

In [12]:
library(withr)
overlap_factor <- 5
contingency_data$mark <- contingency_data$mark %>%
  factor(str_glue("H3K{c(4,27,9)}me3"))
contingency_test <- contingency_data %>%
  group_by(mark, region) %>%
  reframe(
    p.adjust = mean(
      {
        p <- table[1, 1] / (table[1, 1] + table[2, 2])
        q <- table[2, 2] / (table[1, 1] + table[2, 2])
        n <- round((table[1, 1] + table[2, 2]) / overlap_factor)
        if (table[1, 1] < table[2, 2]) {
          lower <- table[1, 1] / overlap_factor
          upper <- table[2, 2] / overlap_factor
        } else {
          lower <- table[2, 2] / overlap_factor
          upper <- table[1, 1] / overlap_factor
        }
        pbinom(upper - 1, n, prob = 0.5, lower.tail = F) +
          pbinom(lower, n, prob = 0.5)
      }
    )
  ) %>%
  mutate(
    p.adjust = p.adjust %>% p.adjust(),
    signif = structure(
      cut(p.adjust, c(-Inf, 1e-4, 1e-3, 1e-2, 5e-2, Inf)),
      levels = c("****", "***", "**", "*", ""),
      class = "factor"
    )
  )
contingency_test

mark,region,p.adjust,signif
<fct>,<fct>,<dbl>,<fct>
H3K4me3,2L,5.962244e-06,****
H3K4me3,2LC,1.048502e-11,****
H3K4me3,2RC,3.074202e-19,****
H3K4me3,2R,3.342499e-01,
H3K4me3,3L,1.583402e-20,****
H3K4me3,3LC,1.158653e-18,****
H3K4me3,3RC,1.041905e-05,****
H3K4me3,3R,4.709798e-05,****
H3K4me3,4,8.665765e-25,****
